In [4]:
!pip install -r requirements.txt

In [5]:
import numpy as np
from tqdm import tqdm
import transformers
from transformers import (
    BertTokenizer,
    BertForMaskedLM
)
import pandas as pd
import torch
from torch.nn import functional as F
from transformers import logging

logging.set_verbosity_error()

In [6]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased', return_dict=True)

In [7]:
import gensim
from gensim import downloader
from gensim.models import KeyedVectors
# word2vec = gensim.downloader.load('glove-wiki-gigaword-300')
# word2vec.save('vectors.kv')
# word2vec = KeyedVectors.load('vectors.kv')
# print(type(word2vec.key_to_index))
# print(len(word2vec.key_to_index))
# VOCABULARY = list(word2vec.key_to_index.keys())

In [167]:
# original word embeddings from BERT
print(VOCABULARY[0])
print(len(VOCABULARY))

the
400000


In [8]:
NULL_PROJECTION = np.load("data/nullspace_vector.npy")

In [190]:
with open("data/embeddings/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings.txt", "r") as f:
    lines = f.readlines()
lines = lines[1:]

In [20]:
from collections import defaultdict

In [192]:
d = defaultdict(int)
for line in lines:
    split = line.split()
    key = split[0]
    value = split[1:]
    value = np.array(list(map(float, value)))
    d[key] = value

In [193]:
print(len(d))

9794


In [194]:
# debiased vectors
for key, value in d.items():   
    d[key] = NULL_PROJECTION.dot(value.T).T
# import numpy as np
# m = gensim.models.keyedvectors.Word2VecKeyedVectors(vector_size=768)


In [198]:
with open("data/embeddings/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings-debiased.txt", "w") as bert_file:
    bert_file.write("9794 768\n")
    for key, value in d.items():
        bert_file.write(f"{key} {' '.join(map(str, list(value)))}\n")
    # for word, embedding in zip(VOCABULARY, bert_vocab_embedding_list):
        

In [11]:
# for key,value in d.items():   
    # m.add_vectors(key, np.array(value))

In [40]:
def load_word_vectors(fname):
    model = KeyedVectors.load_word2vec_format(fname, binary=False)
    vecs = model.vectors
    words = list(model.index_to_key)
    return model, vecs, words

BASE_PATH = "data/embeddings"
EMBEDDINGS_FILE_PATH = f"{BASE_PATH}/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings.txt"

embed, vecs, words = load_word_vectors(fname=EMBEDDINGS_FILE_PATH)
# print(len(embed["the"]))

In [39]:
BASE_PATH = "data/embeddings"
DEBIASED_EMBEDDINGS_FILE_PATH = f"{BASE_PATH}/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings-debiased.txt"

debiased_embed, _,_= load_word_vectors(fname=DEBIASED_EMBEDDINGS_FILE_PATH)

In [11]:
print(len(words))

9794


In [12]:
def cos_sim(a,b):
    # a, b: numpy.ndarray
    return np.dot(a,b) / (np.linalg.norm(a)*np.linalg.norm(b))

print(cos_sim(embed["she"],embed["doctor"]))

0.7119591


In [13]:
# DIRECT-BIAS
# c: parameter that determines how strict do we want to in measuring bias
# w: the word embedding vector
# theta: the gender direction 
# N: the set of gender neutral words

def direct_bias_debiased(gender_neutral_words, theta):
    # gender_neutral_words: list of gender neutral words
    # ["doctor", "teacher",...]
    # theta: nd.array
    # N = len(gender_neutral_words)
    N = 0
    bias = 0
    for w in gender_neutral_words:
        if w in debiased_embed.key_to_index:
            N += 1
            bias += cos_sim(theta, debiased_embed[w])
    bias /= N
    return bias

def direct_bias(gender_neutral_words, theta):
    # gender_neutral_words: list of gender neutral words
    # ["doctor", "teacher",...]
    # theta: nd.array
    # N = len(gender_neutral_words)
    N = 0
    bias = 0
    for w in gender_neutral_words:
        if w in embed.key_to_index:
            N += 1
            bias += cos_sim(theta, embed[w])
    bias /= N
    return bias

In [14]:
PAIRS = [["woman", "man"], ["girl", "boy"], ["she", "he"], ["mother", "father"], ["daughter", "son"], ["female", "male"], ["her", "his"], ["herself", "himself"], ["mary", "john"]]
NUM_PCA_COMPONENTS=10
from sklearn.decomposition import PCA

def doPCA(pairs, num_components=10):
    matrix = []
    for a, b in pairs:
        center = (a + b) / 2
        norm_a = a - center
        norm_b = b - center
        matrix.append(norm_a)
        matrix.append(norm_b)
    matrix = np.array(matrix)
    pca = PCA(n_components=num_components, svd_solver="full")
    pca.fit(matrix)
    return pca

def bias_subspace(embed, num_components=None):
    # define gender direction
    pair = []
    for p in PAIRS:
        pair.append((embed[p[0]], embed[p[1]]))
    pca = doPCA(pair)
    bias_direction = pca.components_[0] # using principle component that has the highest variance in gender bias subspace
    print(f"pca explained variance ratio: {pca.explained_variance_ratio_} \n")
    
    return bias_direction

bias_direction = bias_subspace(embed=embed, num_components=NUM_PCA_COMPONENTS)

bias_sign = bias_direction.dot(embed["woman"] - embed["man"])
bias_direction = -bias_direction if bias_sign > 0 else bias_direction

pca explained variance ratio: [2.39618391e-01 2.31649175e-01 1.17617190e-01 1.07486375e-01
 8.77610967e-02 6.57494664e-02 5.98696880e-02 5.55527359e-02
 3.46959345e-02 5.94005614e-15] 



In [15]:
# direct-bias of the whole vocabulary before debiasing
direct_bias(words, bias_direction)


-0.0342985351735214

In [16]:
# direct-bias of the whole vocabulary after debiasing
direct_bias_debiased(words, bias_direction)

0.0002429442899980663

In [209]:
def indirect_bias(w, v, theta):
# w,v: two word vectors np.array
# theta: gender direction np.array
    bias = 0
    w_perp = theta *(np.dot(w,theta) / (np.linalg.norm(theta)*np.linalg.norm(theta)))
    v_perp = theta *(np.dot(v,theta) / (np.linalg.norm(theta)*np.linalg.norm(theta)))
    bias = np.dot(w,v) - np.dot(w_perp,v_perp)/np.linalg.norm(w_perp)/np.linalg.norm(v_perp)
    bias /= np.dot(w,v)
    return bias

In [210]:
print(indirect_bias(embed["she"], embed["doctor"], bias_direction))
print(indirect_bias(embed["she"], embed["nurse"], bias_direction))
print(indirect_bias(embed["she"], embed["person"], bias_direction))

0.99320835
0.9929431
0.99346185


In [211]:
print(indirect_bias(embed["she"], embed["waiter"], bias_direction))
print(indirect_bias(embed["she"], embed["waitress"], bias_direction))
print(indirect_bias(embed["he"], embed["waiter"], bias_direction))
print(indirect_bias(embed["he"], embed["waitress"], bias_direction))

KeyError: "Key 'waiter' not present"

In [ ]:
# INDIRECT-BIAS
# def indirect_bias(gender_neutral_words, theta):
#     # gender_neutral_words: list of gender neutral words
#     # ["doctor", "teacher",...]
#     # theta: nd.array
#     N = len(gender_neutral_words)
#     bias = 0
#     for w in gender_neutral_words:
#         bias += cos_sim(theta, embed[w])
#     bias /= N
#     return bias

In [17]:
# natural gender word
with open("natural_gendered_vocab.txt") as f:
    natural_gender_vocab  = f.readlines()
for idx, word in enumerate(natural_gender_vocab):
    natural_gender_vocab[idx] = word[0:-1]


In [ ]:
# vocab_withou_natural_gendered_words = 

In [48]:
with open("data/embeddings/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings.txt", "r") as f:
    lines = f.readlines()
lines = lines[1:]

In [49]:
d_without_natural_gendered_words = defaultdict(int)
for line in lines:
    split = line.split()
    key = split[0]
    value = split[1:]
    value = np.array(list(map(float, value)))
    d_without_natural_gendered_words[key] = value

In [50]:
count = 0
for key, value in d_without_natural_gendered_words.items(): 
    if key in natural_gender_vocab:
        count += 1
    else:
    # if key not in natural_gender_vocab:
        d_without_natural_gendered_words[key] = NULL_PROJECTION.dot(value.T).T

print(count)

81


In [51]:
with open("data/embeddings/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings-debiased-without-natural-gender.txt", "w") as bert_file:
    bert_file.write("9794 768\n")
    for key, value in d_without_natural_gendered_words.items():
        bert_file.write(f"{key} {' '.join(map(str, list(value)))}\n")

In [52]:
BASE_PATH = "data/embeddings"
DEBIASED_EMBEDDINGS_FILE_PATH = f"{BASE_PATH}/BERTLM_ENCODER_LAYER_ONE/bert-base-uncased-embeddings-debiased-without-natural-gender.txt"

debiased_embed_without_natural_gender, _, _= load_word_vectors(fname=DEBIASED_EMBEDDINGS_FILE_PATH)

In [36]:
def direct_bias_debiased_wo_natural_gender(gender_neutral_words, theta):
    # gender_neutral_words: list of gender neutral words
    # ["doctor", "teacher",...]
    # theta: nd.array
    # N = len(gender_neutral_words)
    N = 0
    bias = 0
    for w in gender_neutral_words:
        if w in debiased_embed_without_natural_gender.key_to_index:
            N += 1
            bias += cos_sim(theta, debiased_embed_without_natural_gender[w])
    bias /= N
    return bias

In [37]:
# print(words)
# print(debiased_embed_without_natural_gender.key_to_index)
direct_bias_debiased_wo_natural_gender(words, bias_direction)

-4.279754778187711e-05

In [222]:

embed.most_similar(positive=['girl'])

[('guy', 0.8692551255226135),
 ('boy', 0.8468469977378845),
 ('woman', 0.8374586701393127),
 ('girlfriend', 0.8256111145019531),
 ('kid', 0.8209311366081238),
 ('nickname', 0.7994575500488281),
 ('man', 0.7934375405311584),
 ('chick', 0.7893049120903015),
 ('teenage', 0.7892447710037231),
 ('banana', 0.7854103446006775)]

In [223]:
embed.most_similar(positive=['she'])

[('he', 0.9345009326934814),
 ('i', 0.8788574934005737),
 ('her', 0.8645201921463013),
 ('cookie', 0.857197642326355),
 ('cradle', 0.8511320352554321),
 ('lou', 0.8493645787239075),
 ('extremely', 0.8493454456329346),
 ('roommate', 0.8489691615104675),
 ('pan', 0.8487756848335266),
 ('hunting', 0.8482562303543091)]

In [224]:
embed.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7754306197166443),
 ('princess', 0.7402370572090149),
 ('prince', 0.7396592497825623),
 ('spouse', 0.7351313233375549),
 ('wife', 0.7235236763954163),
 ('lion', 0.719057023525238),
 ('crown', 0.7158281803131104),
 ('partner', 0.7101830840110779),
 ('knight', 0.7070295810699463),
 ('noble', 0.7017359733581543)]

In [225]:

debiased_embed.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.5136861205101013),
 ('kings', 0.5069079399108887),
 ('prince', 0.43332844972610474),
 ('princess', 0.4071042537689209),
 ('spouse', 0.3630116879940033),
 ('lion', 0.3624734878540039),
 ('governor', 0.348253458738327),
 ('lady', 0.34707921743392944),
 ('husband', 0.3319151997566223),
 ('heroes', 0.3124328851699829)]

In [226]:
# word analogy test
debiased_embed_without_natural_gender.most_similar(positive=['woman', 'king'], negative=['man'])
debiased_embed_without_natural_gender.most_similar(positive=['woman', 'king'], negative=['man'])[0][0]

'queen'

In [227]:
# word analogy test using a dataset

# preserving the original semantics
with open("questions-words.txt") as f:
    q_ws = f.readlines()
count = 0
sim_sum = 0
for q_w in q_ws:
    q_w = q_w.split()
    if len(q_w) == 4:
        word1, word2, word3, word4 = q_w   
        if (word1 in embed.key_to_index) & (word2 in embed.key_to_index) & (word3 in embed.key_to_index) & (word4 in embed.key_to_index):
            count += 1
            a = embed.most_similar(positive=[word1, word4], negative=[word3])[0][0]
            sim_sum += cos_sim(embed[a], embed[word2])

sim_sum /= count
print(sim_sum)

0.8732956366921892


In [228]:
print(count)

3020


In [229]:
print(len(q_ws))

19558


In [230]:
with open("questions-words.txt") as f:
    q_ws = f.readlines()
count = 0
sim_sum = 0
for q_w in q_ws:
    q_w = q_w.split()
    if len(q_w) == 4:
        word1, word2, word3, word4 = q_w   
        if (word1 in debiased_embed_without_natural_gender.key_to_index) & (word2 in debiased_embed_without_natural_gender.key_to_index) & (word3 in debiased_embed_without_natural_gender.key_to_index) & (word4 in debiased_embed_without_natural_gender.key_to_index):
            count += 1
            a = debiased_embed_without_natural_gender.most_similar(positive=[word1, word4], negative=[word3])[0][0]
            sim_sum += cos_sim(debiased_embed_without_natural_gender[a], debiased_embed_without_natural_gender[word2])

sim_sum /= count
print(count)
print(sim_sum)

3020
0.705818334265168


In [231]:
with open("questions-words.txt") as f:
    q_ws = f.readlines()
count = 0
sim_sum = 0
for q_w in q_ws:
    q_w = q_w.split()
    if len(q_w) == 4:
        word1, word2, word3, word4 = q_w   
        if (word1 in debiased_embed.key_to_index) & (word2 in debiased_embed.key_to_index) & (word3 in debiased_embed.key_to_index) & (word4 in debiased_embed.key_to_index):
            count += 1
            a = debiased_embed.most_similar(positive=[word1, word4], negative=[word3])[0][0]
            sim_sum += cos_sim(debiased_embed[a], debiased_embed[word2])

sim_sum /= count
print(count)
print(sim_sum)

3020
0.7184189895150865


In [ ]:
discriminator.load_state_dict(torch.load(f"generator_0.pt"))

In [41]:
embed.most_similar(positive=['she'])

[('he', 0.9345009326934814),
 ('i', 0.8788574934005737),
 ('her', 0.8645201921463013),
 ('cookie', 0.857197642326355),
 ('cradle', 0.8511320352554321),
 ('lou', 0.8493645787239075),
 ('extremely', 0.8493454456329346),
 ('roommate', 0.8489691615104675),
 ('pan', 0.8487756848335266),
 ('hunting', 0.8482562303543091)]

In [42]:
debiased_embed.most_similar(positive=['she'])

[('he', 0.781117856502533),
 ('it', 0.4479445815086365),
 ('i', 0.43707892298698425),
 ('they', 0.43589672446250916),
 ('her', 0.3615175187587738),
 ('never', 0.3547224700450897),
 ('was', 0.3250300884246826),
 ('roommate', 0.3198656439781189),
 ('truly', 0.3180709183216095),
 ('what', 0.3171789348125458)]

In [57]:
debiased_embed_without_natural_gender.most_similar(positive=['she'])

[('he', 0.9345009326934814),
 ('her', 0.8645201921463013),
 ('dean', 0.847304105758667),
 ('cow', 0.8209158778190613),
 ('girlfriend', 0.8096147775650024),
 ('his', 0.807951033115387),
 ('bride', 0.7995162010192871),
 ('cowboy', 0.7987228035926819),
 ('sister', 0.797659695148468),
 ('ordinary', 0.7925541400909424)]

In [44]:
embed.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7754306197166443),
 ('princess', 0.7402370572090149),
 ('prince', 0.7396592497825623),
 ('spouse', 0.7351313233375549),
 ('wife', 0.7235236763954163),
 ('lion', 0.719057023525238),
 ('crown', 0.7158281803131104),
 ('partner', 0.7101830840110779),
 ('knight', 0.7070295810699463),
 ('noble', 0.7017359733581543)]

In [45]:
debiased_embed.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.5136861205101013),
 ('kings', 0.5069079399108887),
 ('prince', 0.43332844972610474),
 ('princess', 0.4071042537689209),
 ('spouse', 0.3630116879940033),
 ('lion', 0.3624734878540039),
 ('governor', 0.348253458738327),
 ('lady', 0.34707921743392944),
 ('husband', 0.3319151997566223),
 ('heroes', 0.3124328851699829)]

In [55]:
debiased_embed_without_natural_gender.most_similar(positive=['woman', 'king'], negative=['man'])

[('queen', 0.7754306197166443),
 ('princess', 0.7402370572090149),
 ('wife', 0.7235236763954163),
 ('husband', 0.7009216547012329),
 ('lady', 0.6997972130775452),
 ('priest', 0.684046745300293),
 ('girlfriend', 0.6816965937614441),
 ('count', 0.6771528124809265),
 ('daughter', 0.675297737121582),
 ('duke', 0.6748501658439636)]